<a href="https://colab.research.google.com/github/thunguyen177/DPER/blob/main/DPER_multiple_class_without_equal_covariance_assumption__MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## libraries and function 

In [ ]:
!pip install impyute
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as skLDA
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from scipy import stats
import numpy as np
import impyute as impy
from fancyimpute import IterativeSVD, SoftImpute, NuclearNormMinimization
import pandas as pd
import time 
from missingpy import MissForest

distutils: /home/thu/.conda/envs/tf2/include/python3.8/UNKNOWN
sysconfig: /home/thu/.conda/envs/tf2/include/python3.8
user = False
home = None
root = None
prefix = None
distutils: /home/thu/.conda/envs/tf2/include/python3.8/UNKNOWN
sysconfig: /home/thu/.conda/envs/tf2/include/python3.8
user = False
home = None
root = None
prefix = None
You should consider upgrading via the '/home/thu/.conda/envs/tf2/bin/python -m pip install --upgrade pip' command.


/home/thu/.conda/envs/tf2/lib/python3.8/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


### MLE estimation function 

In [ ]:
def diag_term(X,i):
  arr0 = X[:,i].flatten()
  arr = arr0[~np.isnan(arr0)]
  return np.var(arr)

def musMLE(X,y,G):
    n,p = X.shape[0], X.shape[1]
    f = lambda g: np.nanmean(X[y==g,:],axis=0)
    musMLE = np.array([f(g) for g in range(G)])    
    return musMLE.T

def Smle(X,y,musMLE,g):
    '''
    function to compute the covariance matrix for the g-th class
    X: input, should be a numpy array
    y: label
    G: number of classes
    g: class index
    output:
    - mus: each row is a class mean
    - S: common covariance matrix of class 1,2,..., G 
    '''
    epsilon = 1e-5 # define epsilon to put r down to 0 if r < epsilon
    Xg, yg = X[y==g,:], y[y==g]
    n,p = Xg.shape[0], Xg.shape[1] 
 
    S = np.diag([diag_term(Xg,i) for i in range(p)]) 

    for i in range(p):      
      for j in range(i):
        if ((S[i,i] == 0.) | (S[j,j] == 0.)):
          S[i,j] = S[j,i] = 0.
          continue

        mat = Xg[:,[i,j]]

        # drop rows with NA
        idx = ~np.isnan(mat).any(axis=1)
        mat, y_arr = mat[idx], yg[idx]
        A = mg = len(y_arr) 

        s11 = mg*np.var(mat[:,0])
        s22 = mg*np.var(mat[:,1])
        s12 = sum((mat[:,0]-musMLE[i,g])*(mat[:,1]-musMLE[j,g]))
        B = S[i,i]*S[j,j]*A - s22 * S[i,i] - s11 * S[j,j]
        coefficient = [-A, s12, B, s12*S[i,i]*S[j,j]]
        r = np.roots(coefficient)
        r = r[abs(np.imag(r)) < epsilon]
        r = np.real(r)
        r[abs(r) < epsilon] = 0
 
        if len(r)>1:
          condi_var = S[j,j] - r**2/S[i,i]
          eta = -A*np.log(condi_var)-(S[j,j]-2*r/S[i,i]*s12 + r**2/S[i,i]**2*s11)/condi_var
          r = r[eta == max(eta[~np.isnan(eta)])]

        if len(r) > 1: 
          if sum(r==0.0) == len(r):
            r = 0.
          else:  
            w = np.cov(mat, rowvar=False)  
            #r = r[w[0,1]*r>=0]
            r = r[np.abs(r-w[0,1]).argmin()] # select r that is closet to w[0,1] 

        S[i,j] = S[j,i] = r
    return S

### compute_err function 

In [ ]:


def compute_err(Xtrain, ytrain, G, missing_rate):  
    Xtr_nan = generate_nan(Xtrain, missing_rate)
    
    scaler = StandardScaler()
    scaler.fit(Xtr_nan)
    Xtr_nan = scaler.transform(Xtr_nan)
    Xtrain = scaler.transform(Xtrain)
    
    # estimate parameters from full data
    # each row is a mean of a class
    mus = np.array([np.mean(Xtrain[ytrain==g,:], axis=0) for g in np.arange(G)])
    S = np.array([np.cov(Xtrain[ytrain==g,:],rowvar =False) 
             for g in np.arange(G)])
    print('original mus',mus)
    print('S', S) 

    # MLEs approach
    start = time.time()
    mus_mle = musMLE(Xtr_nan,y,G)
    S_mle = np.array([Smle(Xtr_nan,y,mus_mle, g) for g in range(G)])   
    print('mus MLE',mus_mle)
    print('S MLE', S_mle)          
    mle_err = err(mus, S, mus_mle.T, S_mle)
    mle_time = time.time()-start

    start = time.time()
    Xtr_softimpute = SoftImpute(max_iters = 100).fit_transform(Xtr_nan)
    mus_softimpute = np.asarray([np.mean(Xtr_softimpute[ytrain==g,:], axis=0
                                         ) for g in np.arange(G)])
    S_softimpute = np.asarray([np.cov(Xtr_softimpute[ytrain==g,:], rowvar =False) 
             for g in np.arange(G)])

    print('mus soft impute', mus_softimpute)
    print('S soft impute', S_softimpute)
    softimpute_err =  err(mus, S, mus_softimpute, S_softimpute)
    softimpute_time = time.time()-start

    start = time.time()
    Xtr_mice = IterativeImputer(max_iter=100).fit(Xtr_nan).transform(Xtr_nan)
    mus_mice = np.asarray([np.mean(Xtr_mice[ytrain==g,:], axis=0
                                   ) for g in np.arange(G)])
    S_mice = np.asarray([np.cov(Xtr_mice[ytrain==g,:], rowvar =False) 
             for g in np.arange(G)])

    mice_err = err(mus, S, mus_mice, S_mice)
    mice_time = time.time()-start

    start = time.time()
    Xtr_nuclear = NuclearNormMinimization(max_iters=100).fit_transform(Xtr_nan)
    mus_nuclear = np.asarray([np.mean(Xtr_nuclear[ytrain==g,:], axis=0
                                      ) for g in np.arange(G)])
    S_nuclear = np.asarray([(sum(ytrain==g))*np.cov(Xtr_nuclear[ytrain==g,:], rowvar =False) 
             for g in np.arange(G)])
    nuclear_err = err(mus, S, mus_nuclear, S_nuclear)
    nuclear_time = time.time()-start
    
    start = time.time()
    Xtr_BRidge = IterativeImputer(random_state=0).fit_transform(Xtr_nan)
    mus_BRidge = np.asarray([np.mean(Xtr_BRidge[ytrain==g,:], axis=0
                                      ) for g in np.arange(G)])
    S_BRidge = np.asarray([(sum(ytrain==g))*np.cov(Xtr_BRidge[ytrain==g,:], rowvar =False) 
             for g in np.arange(G)])
    BRidge_err = err(mus, S, mus_BRidge, S_BRidge)
    BRidge_time = time.time()-start

    start = time.time()
    Xtr_mforest = MissForest(random_state=0).fit_transform(Xtr_nan)
    mus_mforest = np.asarray([np.mean(Xtr_mforest[ytrain==g,:], axis=0
                                      ) for g in np.arange(G)])
    S_mforest = np.asarray([(sum(ytrain==g))*np.cov(Xtr_mforest[ytrain==g,:], rowvar =False) 
             for g in np.arange(G)])
    S_mforest = np.sum(S_mforest, axis = 0)/len(ytrain)
    mforest_err = err(mus, S, mus_mforest, S_mforest)
    mforest_time = time.time()-start      
    err_rate = np.vstack((mle_err, mice_err, softimpute_err,nuclear_err, BRidge_err))
    return err_rate

## Individual compute error function 

In [ ]:
def err(mus, S, mus_est, S_est):
  er = [np.linalg.norm(mus_est-mus)/mus.size,
        np.linalg.norm(S_est.flatten().flatten()-S.flatten())/S.size]  
  return np.mean(er)  

def generate_nan(Xtrain, missing_rate):
  Xshape = Xtrain.shape
  na_id = np.random.randint(0,Xtrain.size,round(missing_rate*Xtrain.size))
  Xtr_nan = Xtrain.flatten()
  Xtr_nan[na_id] = np.nan 
  return Xtr_nan.reshape(Xshape) 

In [ ]:
def compute_err_dper(Xtrain, ytrain, G, missing_rate):  
    Xtr_nan = generate_nan(Xtrain, missing_rate)
    
    scaler = StandardScaler()
    scaler.fit(Xtr_nan)
    Xtr_nan = scaler.transform(Xtr_nan)
    Xtrain = scaler.transform(Xtrain)
    
    # estimate parameters from full data
    # each row is a mean of a class
    mus = np.array([np.mean(Xtrain[ytrain==g,:], axis=0) for g in np.arange(G)])
    S = np.array([np.cov(Xtrain[ytrain==g,:],rowvar =False) 
             for g in np.arange(G)]) 

    start = time.time()
    mus_mle = musMLE(Xtr_nan,y,G)
    S_mle = np.array([Smle(Xtr_nan,y,mus_mle, g) for g in range(G)])   
    mle_err = err(mus, S, mus_mle.T, S_mle)
    mle_time = time.time()-start
    
    return mle_err, mle_time

In [ ]:
def compute_err_soft(Xtrain, ytrain, G, missing_rate):  
    Xtr_nan = generate_nan(Xtrain, missing_rate)
    
    scaler = StandardScaler()
    scaler.fit(Xtr_nan)
    Xtr_nan = scaler.transform(Xtr_nan)
    Xtrain = scaler.transform(Xtrain)
    
    # estimate parameters from full data
    # each row is a mean of a class
    mus = np.array([np.mean(Xtrain[ytrain==g,:], axis=0) for g in np.arange(G)])
    S = np.array([np.cov(Xtrain[ytrain==g,:],rowvar =False) 
             for g in np.arange(G)]) 

    start = time.time()
    Xtr_softimpute = SoftImpute(max_iters = 100).fit_transform(Xtr_nan)
    mus_softimpute = np.asarray([np.mean(Xtr_softimpute[ytrain==g,:], axis=0
                                         ) for g in np.arange(G)])
    S_softimpute = np.asarray([np.cov(Xtr_softimpute[ytrain==g,:], rowvar =False) 
             for g in np.arange(G)])

    softimpute_err =  err(mus, S, mus_softimpute, S_softimpute)
    softimpute_time = time.time()-start
 
    return softimpute_err, softimpute_time

In [ ]:
def compute_err_mice(Xtrain, ytrain, G, missing_rate):  
    Xtr_nan = generate_nan(Xtrain, missing_rate)
    
    scaler = StandardScaler()
    scaler.fit(Xtr_nan)
    Xtr_nan = scaler.transform(Xtr_nan)
    Xtrain = scaler.transform(Xtrain)
    
    # estimate parameters from full data
    # each row is a mean of a class
    mus = np.array([np.mean(Xtrain[ytrain==g,:], axis=0) for g in np.arange(G)])
    S = np.array([np.cov(Xtrain[ytrain==g,:],rowvar =False) 
             for g in np.arange(G)]) 

    start = time.time()
    Xtr_mice = IterativeImputer(max_iter=100).fit(Xtr_nan).transform(Xtr_nan)
    mus_mice = np.asarray([np.mean(Xtr_mice[ytrain==g,:], axis=0
                                   ) for g in np.arange(G)])
    S_mice = np.asarray([np.cov(Xtr_mice[ytrain==g,:], rowvar =False) 
             for g in np.arange(G)])

    mice_err = err(mus, S, mus_mice, S_mice)
    mice_time = time.time()-start
 
    return mice_err, mice_time

In [ ]:
def compute_err_nuclear(Xtrain, ytrain, G, missing_rate):  
    Xtr_nan = generate_nan(Xtrain, missing_rate)
    
    scaler = StandardScaler()
    scaler.fit(Xtr_nan)
    Xtr_nan = scaler.transform(Xtr_nan)
    Xtrain = scaler.transform(Xtrain)
    
    # estimate parameters from full data
    # each row is a mean of a class
    mus = np.array([np.mean(Xtrain[ytrain==g,:], axis=0) for g in np.arange(G)])
    S = np.array([np.cov(Xtrain[ytrain==g,:],rowvar =False) 
             for g in np.arange(G)]) 

    start = time.time()
    Xtr_nuclear = NuclearNormMinimization(max_iters=100).fit_transform(Xtr_nan)
    mus_nuclear = np.asarray([np.mean(Xtr_nuclear[ytrain==g,:], axis=0
                                      ) for g in np.arange(G)])
    S_nuclear = np.asarray([np.cov(Xtr_nuclear[ytrain==g,:], rowvar =False) 
             for g in np.arange(G)])
    nuclear_err = err(mus, S, mus_nuclear, S_nuclear)
    nuclear_time = time.time()-start
 
    return nuclear_err, nuclear_time

In [ ]:
def compute_err_mforest(Xtrain, ytrain, G, missing_rate):  
    Xtr_nan = generate_nan(Xtrain, missing_rate)
    
    scaler = StandardScaler()
    scaler.fit(Xtr_nan)
    Xtr_nan = scaler.transform(Xtr_nan)
    Xtrain = scaler.transform(Xtrain)
    
    # estimate parameters from full data
    # each row is a mean of a class
    mus = np.array([np.mean(Xtrain[ytrain==g,:], axis=0) for g in np.arange(G)])
    S = np.array([np.cov(Xtrain[ytrain==g,:],rowvar =False) 
             for g in np.arange(G)]) 

    start = time.time()
    Xtr_mforest = MissForest(random_state=0).fit_transform(Xtr_nan)
    mus_mforest = np.asarray([np.mean(Xtr_mforest[ytrain==g,:], axis=0
                                      ) for g in np.arange(G)])
    S_mforest = np.asarray([np.cov(Xtr_mforest[ytrain==g,:], rowvar =False) 
             for g in np.arange(G)])
    mforest_err = err(mus, S, mus_mforest, S_mforest)
    mforest_time = time.time()-start      
 
    return mforest_err, mforest_time

### MNIST

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Fetch the dataset directly
mnist = tfds.image.MNIST()
# or by string name
mnist = tfds.builder('mnist')

# Download the data, prepare it, and write it to disk
mnist.download_and_prepare()

# Load data from disk as tf.data.Datasets
datasets = mnist.as_dataset()
train_dataset, test_dataset = datasets['train'], datasets['test']

In [ ]:
# convert the Dataset to NumPy arrays and flatten the data
Xtrain, ytrain = [], []
for example in tfds.as_numpy(train_dataset):
  Xtrain.append(example['image'].flatten())
  ytrain.append(example['label'])

Xtrain, ytrain = np.asarray(Xtrain), np.asarray(ytrain)
Xtrain = Xtrain.astype(float)

# convert the test set to NumPy arrays and flatten the data
Xtest, ytest = [], []
for example in tfds.as_numpy(test_dataset):
  Xtest.append(example['image'].flatten())
  ytest.append(example['label'])

Xtest, ytest = np.asarray(Xtest), np.asarray(ytest)
Xtest = Xtest.astype(float)
X = np.vstack((Xtrain,Xtest))
y = np.hstack((ytrain,ytest))
X.shape, y.shape
# set random seed and shuffle the data
np.random.seed(1)
idx = np.arange(len(y))
np.random.shuffle(idx)
X, y = X[idx,:], y[idx]  

X.shape, y.shape 

((70000, 784), (70000,))

In [ ]:
# check if a column is all 0
id = [np.sum(Xtrain[:,i] != 0)>10 for i in range(28**2)]
# number of columns that mostly zero
print(28**2-np.sum(id))
# number of columns that has at least more than 10 non-zero
np.sum(id)
X = X[:, id]

135


In [ ]:
G = 10
np.random.seed(3)
e20_dper = compute_err_dper(X, y, G, 0.2)
e35_dper = compute_err_dper(X, y, G, 0.35)
e50_dper = compute_err_dper(X, y, G, 0.5)
e65_dper = compute_err_dper(X, y, G, 0.65)
e80_dper = compute_err_dper(X, y, G, 0.8)

<ipython-input-2-cba8272b93b5>:54: RuntimeWarning: invalid value encountered in log
  eta = -A*np.log(condi_var)-(S[j,j]-2*r/S[i,i]*s12 + r**2/S[i,i]**2*s11)/condi_var


mus MLE [[-0.01177144 -0.01177144  0.00305949 ... -0.01177144 -0.01177144
  -0.01177144]
 [-0.01429066 -0.01429066  0.01678188 ... -0.01429066 -0.01429066
  -0.01429066]
 [-0.01867879 -0.01867879 -0.00469708 ... -0.01867879 -0.01867879
  -0.01867879]
 ...
 [-0.02359907 -0.02359907 -0.02359907 ...  0.15861852 -0.02359907
   0.02188908]
 [-0.01644835 -0.01644835 -0.01644835 ...  0.07789702 -0.01644835
   0.05062416]
 [-0.01097203 -0.01097203 -0.01097203 ...  0.04767043 -0.01097203
   0.03815517]]
S MLE [[[1.20370622e-35 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 3.00926554e-36 0.00000000e+00 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  ...
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.20370622e-35
   0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   0.00000000e+00

<ipython-input-2-cba8272b93b5>:54: RuntimeWarning: invalid value encountered in log
  eta = -A*np.log(condi_var)-(S[j,j]-2*r/S[i,i]*s12 + r**2/S[i,i]**2*s11)/condi_var


mus MLE [[-0.01087208 -0.01087208  0.00505546 ... -0.01087208 -0.01087208
  -0.01087208]
 [-0.01440819 -0.01440819  0.0204351  ... -0.01440819 -0.01440819
  -0.01440819]
 [-0.02037829 -0.02037829 -0.0007683  ... -0.02037829 -0.02037829
  -0.02037829]
 ...
 [-0.02361546 -0.02361546 -0.02361546 ...  0.15076767 -0.02361546
   0.02784092]
 [-0.01502945 -0.01502945 -0.01502945 ...  0.05827425 -0.01502945
   0.0602607 ]
 [-0.01028402 -0.01028402 -0.01028402 ...  0.08680513 -0.01028402
  -0.00729678]]
S MLE [[[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  ...
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 4.81482486e-35
   0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   3.00926554e-36

<ipython-input-2-cba8272b93b5>:54: RuntimeWarning: invalid value encountered in log
  eta = -A*np.log(condi_var)-(S[j,j]-2*r/S[i,i]*s12 + r**2/S[i,i]**2*s11)/condi_var


mus MLE [[-0.00995651 -0.00995651 -0.00770718 ... -0.00995651 -0.00995651
  -0.00995651]
 [-0.01065062 -0.01065062  0.02087705 ... -0.01065062 -0.01065062
  -0.01065062]
 [-0.01979708 -0.01979708  0.00546259 ... -0.01979708 -0.01979708
  -0.01979708]
 ...
 [-0.02465696 -0.02465696 -0.02465696 ...  0.16002026 -0.02465696
   0.03083794]
 [-0.0180411  -0.0180411  -0.0180411  ...  0.09422342 -0.0180411
   0.04676937]
 [-0.01008431 -0.01008431 -0.01008431 ...  0.05368177 -0.01008431
   0.025773  ]]
S MLE [[[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 3.00926554e-36 0.00000000e+00 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  ...
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   1.20370622e-35 

<ipython-input-2-cba8272b93b5>:54: RuntimeWarning: invalid value encountered in log
  eta = -A*np.log(condi_var)-(S[j,j]-2*r/S[i,i]*s12 + r**2/S[i,i]**2*s11)/condi_var


mus MLE [[-0.00890917 -0.00890917 -0.00890917 ... -0.00890917 -0.00890917
  -0.00890917]
 [-0.01344717 -0.01344717  0.00084399 ... -0.01344717 -0.01344717
  -0.01344717]
 [-0.01712241 -0.01712241  0.00889437 ... -0.01712241 -0.01712241
  -0.01712241]
 ...
 [-0.02293446 -0.02293446 -0.02293446 ...  0.11892509 -0.02293446
   0.0578334 ]
 [-0.01864274 -0.01864274 -0.01864274 ...  0.11406476 -0.01864274
   0.02967141]
 [-0.00897224 -0.00897224 -0.00897224 ...  0.01086368 -0.00897224
   0.06000312]]
S MLE [[[ 3.00926554e-36  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  1.20370622e-35  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  1.20370622e-35 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  ...
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  1.20370622e-35
    0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00

<ipython-input-2-cba8272b93b5>:54: RuntimeWarning: invalid value encountered in log
  eta = -A*np.log(condi_var)-(S[j,j]-2*r/S[i,i]*s12 + r**2/S[i,i]**2*s11)/condi_var


mus MLE [[-0.0092276  -0.0092276  -0.0092276  ... -0.0092276  -0.0092276
  -0.0092276 ]
 [-0.01439078 -0.01439078  0.0092594  ... -0.01439078 -0.01439078
  -0.01439078]
 [-0.01608126 -0.01608126 -0.00372639 ... -0.01608126 -0.01608126
  -0.01608126]
 ...
 [-0.02063638 -0.02063638 -0.02063638 ...  0.11573777 -0.02063638
   0.04551383]
 [-0.01890539 -0.01890539 -0.01890539 ...  0.0890405  -0.01890539
   0.05770412]
 [-0.01043645 -0.01043645 -0.01043645 ...  0.01815521 -0.01043645
   0.06511985]]
S MLE [[[ 3.00926554e-36  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  1.20370622e-35 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  ...
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.000

In [ ]:
# the left column is the error, the right column is the running time
np.vstack((e20_dper,e35_dper,e50_dper,e65_dper,e80_dper))

array([[3.78787478e-05, 5.00751978e+03],
       [5.34976667e-05, 3.90330283e+03],
       [6.50925937e-05, 3.09240094e+03],
       [7.94938644e-05, 2.65548703e+03],
       [9.26285194e-05, 2.19216672e+03]])

## SOFT IMPUTE

In [ ]:
e20_soft = compute_err_soft(X, y, G, 0.2)
e35_soft = compute_err_soft(X, y, G, 0.35)
e50_soft = compute_err_soft(X, y, G, 0.5)
e65_soft = compute_err_soft(X, y, G, 0.65)
e80_soft = compute_err_soft(X, y, G, 0.8)

[SoftImpute] Max Singular Value of X_init = 1383.578270
[SoftImpute] Iter 1: observed MAE=0.049855 rank=649
[SoftImpute] Iter 2: observed MAE=0.050241 rank=649
[SoftImpute] Iter 3: observed MAE=0.050551 rank=649
[SoftImpute] Iter 4: observed MAE=0.050775 rank=649
[SoftImpute] Iter 5: observed MAE=0.050911 rank=649
[SoftImpute] Iter 6: observed MAE=0.050965 rank=649
[SoftImpute] Iter 7: observed MAE=0.050953 rank=649
[SoftImpute] Iter 8: observed MAE=0.050897 rank=649
[SoftImpute] Iter 9: observed MAE=0.050819 rank=649
[SoftImpute] Iter 10: observed MAE=0.050736 rank=649
[SoftImpute] Iter 11: observed MAE=0.050658 rank=649
[SoftImpute] Iter 12: observed MAE=0.050592 rank=649
[SoftImpute] Iter 13: observed MAE=0.050537 rank=649
[SoftImpute] Iter 14: observed MAE=0.050494 rank=649
[SoftImpute] Iter 15: observed MAE=0.050461 rank=649
[SoftImpute] Iter 16: observed MAE=0.050436 rank=649
[SoftImpute] Iter 17: observed MAE=0.050417 rank=649
[SoftImpute] Iter 18: observed MAE=0.050403 rank=649

[SoftImpute] Iter 37: observed MAE=0.047530 rank=649
[SoftImpute] Iter 38: observed MAE=0.047528 rank=649
[SoftImpute] Iter 39: observed MAE=0.047527 rank=649
[SoftImpute] Iter 40: observed MAE=0.047526 rank=649
[SoftImpute] Iter 41: observed MAE=0.047525 rank=649
[SoftImpute] Iter 42: observed MAE=0.047525 rank=649
[SoftImpute] Stopped after iteration 42 for lambda=23.863642
mus soft impute [[-0.01290166 -0.014971   -0.01748763 ... -0.02147355 -0.01742848
  -0.00936295]
 [-0.01333382 -0.01272019 -0.01635796 ... -0.02209433 -0.01740409
  -0.01003563]
 [ 0.00665217  0.01843507  0.00974547 ... -0.0269971  -0.01636814
  -0.01182709]
 ...
 [-0.00983727 -0.012444   -0.01806618 ...  0.13582199  0.07783978
   0.05045554]
 [-0.0142952  -0.01526273 -0.01800427 ... -0.02177015 -0.01540266
  -0.01256974]
 [-0.01034623 -0.01255851 -0.01739202 ...  0.03163174  0.04101945
   0.03614778]]
S soft impute [[[ 8.20136300e-03  3.42870430e-03  9.83000467e-04 ... -1.86007978e-05
   -1.82659564e-05  4.487391

[SoftImpute] Max Singular Value of X_init = 890.558525
[SoftImpute] Iter 1: observed MAE=0.044785 rank=649
[SoftImpute] Iter 2: observed MAE=0.044852 rank=649
[SoftImpute] Iter 3: observed MAE=0.044908 rank=649
[SoftImpute] Iter 4: observed MAE=0.044953 rank=649
[SoftImpute] Iter 5: observed MAE=0.044985 rank=649
[SoftImpute] Iter 6: observed MAE=0.045005 rank=649
[SoftImpute] Iter 7: observed MAE=0.045011 rank=649
[SoftImpute] Iter 8: observed MAE=0.045003 rank=649
[SoftImpute] Iter 9: observed MAE=0.044981 rank=649
[SoftImpute] Iter 10: observed MAE=0.044946 rank=649
[SoftImpute] Iter 11: observed MAE=0.044897 rank=649
[SoftImpute] Iter 12: observed MAE=0.044835 rank=649
[SoftImpute] Iter 13: observed MAE=0.044761 rank=649
[SoftImpute] Iter 14: observed MAE=0.044677 rank=649
[SoftImpute] Iter 15: observed MAE=0.044584 rank=649
[SoftImpute] Iter 16: observed MAE=0.044483 rank=649
[SoftImpute] Iter 17: observed MAE=0.044377 rank=649
[SoftImpute] Iter 18: observed MAE=0.044266 rank=649


[SoftImpute] Iter 3: observed MAE=0.042637 rank=649
[SoftImpute] Iter 4: observed MAE=0.042664 rank=649
[SoftImpute] Iter 5: observed MAE=0.042685 rank=649
[SoftImpute] Iter 6: observed MAE=0.042699 rank=649
[SoftImpute] Iter 7: observed MAE=0.042707 rank=649
[SoftImpute] Iter 8: observed MAE=0.042707 rank=649
[SoftImpute] Iter 9: observed MAE=0.042700 rank=649
[SoftImpute] Iter 10: observed MAE=0.042687 rank=649
[SoftImpute] Iter 11: observed MAE=0.042665 rank=649
[SoftImpute] Iter 12: observed MAE=0.042637 rank=649
[SoftImpute] Iter 13: observed MAE=0.042602 rank=649
[SoftImpute] Iter 14: observed MAE=0.042559 rank=649
[SoftImpute] Iter 15: observed MAE=0.042510 rank=649
[SoftImpute] Iter 16: observed MAE=0.042455 rank=649
[SoftImpute] Iter 17: observed MAE=0.042395 rank=649
[SoftImpute] Iter 18: observed MAE=0.042329 rank=649
[SoftImpute] Iter 19: observed MAE=0.042258 rank=649
[SoftImpute] Iter 20: observed MAE=0.042183 rank=649
[SoftImpute] Iter 21: observed MAE=0.042105 rank=649


In [ ]:
# the left column is the error, the right column is the running time
np.vstack((e20_soft, e35_soft,e50_soft,e65_soft,e80_soft))

array([[5.06105395e-05, 1.73117715e+03],
       [9.29168593e-05, 2.17783741e+03],
       [1.41229975e-04, 2.87814093e+03],
       [1.95521524e-04, 3.45418104e+03],
       [2.53988216e-04, 4.21292992e+03]])

## MICE IMPUTE

In [ ]:
# run from 10pm to 7:20 AM without giving a result
e20_mice = compute_err_mice(X, y, G, 0.2)

In [ ]:
e35_ice = compute_err_mice(X, y, G, 0.35)

In [ ]:
e50_mice = compute_err_mice(X, y, G, 0.5)

In [ ]:
e65_mice = compute_err_mice(X, y, G, 0.65)

In [ ]:
e80_mice = compute_err_mice(X, y, G, 0.8)

## Nuclear norm

In [ ]:
#kernel dies
e20_nuclear = compute_err_nuclear(X, y, G, 0.2)

In [ ]:
#kernel dies
e35_nuclear = compute_err_nuclear(X, y, G, 0.35)

In [ ]:
#kernel dies
e50_nuclear = compute_err_nuclear(X, y, G, 0.5)

In [ ]:
#kernel dies
e65_nuclear = compute_err_nuclear(X, y, G, 0.65)

In [ ]:
#kernel dies
e80_nuclear = compute_err_nuclear(X, y, G, 0.8)

## Miss forest

In [ ]:
#start_time: 2:37pm. interupt time: 10:27 pm
G = 10
e20_mforest = compute_err_mforest(X, y, G, 0.2)

In [ ]:
#start_time: 11:57 am. interupt time: 9:50pm
G = 10
compute_err_mforest(X, y, G, 0.35)

In [ ]:
#start_time: 9:50 pm. interupt time: 7:41 am
G = 10
compute_err_mforest(X, y, G, 0.5)

In [ ]:
#start_time: 7:41 am. interupt time: 1:01 pm
G = 10
compute_err_mforest(X, y, G, 0.65)

In [ ]:
#start_time: 10:29pm. interupt time: 7:41 am
G = 10
compute_err_mforest(X, y, G, 0.8)